# Network analysis in Senegal

### Objectives
    1)	Use measures of road-based accessibility to identify road segments that, if rehabilitated, would improve agricultural market activities in Senegal, including during flood conditions.
    2)	Gain a better understanding of the accessibility, connectivity, and criticality of roads in Senegal in relationship to agricultural origins, processing & transfer sites, and markets.

To this end, the team will develop an accessibility model which measures the travel time from sites of agricultural production to their nearest populated areas, processing centers, and markets. 

### Datasets for analysis
#### ORIGIN
    1) agriculture: MapSPAM 2017. Measuring value in international dollars.
    2) agriculture: UMD Land Cover 2019 30m. Assign MapSPAM value onto land cover cropland class for more precise origin information.
    3) population: WorldPop 2020, UN-adjusted.
    4) settlement extent: GRID3 2020.
#### DESTINATION
    4) markets: derived from WorldPop 2020 and GRID3 2020 urban clusters.
    5) agricultural processing hubs: to be acquired.
#### TRAVEL ROUTE
    6) roads: OpenStreetMap, July 2021.
    7) elevation: 
#### OBSTACLE
    8) flood: FATHOM. 1-in-10, 20, and 50 year flood return periods. These are combined pluvial and fluvial (undefended) flood layers whereby whichever flood level was higher was retained.
#### INTERVENTION
    9) upcoming road projects: AGEROUTE interventions separate from the World Bank-financed project
    10) targeted road projects: critical road segments identified by this accessibility model's baseline outputs


### Model design
#### Basic formula: 
    (a) Off-road driving time from origin to closest road node
    +
    (b) Driving time from road node in (a) to a destination (closeness measured by road segments speeds)

#### Model origin & destination (OD) sets:
    A)	Travel time from an area that has agricultural value/potential to the nearest processing hub (if provided).
    B)	Travel time from an area that has agricultural value/potential to the nearest larger settlement, (“larger” settlement identified using a case-appropriate population metric to be determined).
    C)	Travel time from an area that has agricultural value/potential to the nearest market.
    D)	Travel time from all settlements to the nearest market.
    E)	Travel time from larger settlements to the nearest market.

#### Before/after scenarios for each OD set:
    1)	Pre-project, baseline weather: No inclement weather. Road network status as of November 2021.
    2)	Pre-project, flood: 1-in-10, 1-in-20 and 1-in-50 year flood return period. Road network status as of November 2021.
    3)	Post-project, baseline weather: No inclement weather. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).
    4)	Post-project, flood: 1-in-10 year flood return period. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).

#### Notes:
    --Destinations are expected to be proximal to the road network, so no measure is taken between road and destination.
    --All travel times will be assigned to each model variation’s point of origin; the aggregation up to admin areas is possible if desired.
    --Obstacles & interventions modify the road segment speeds. Basic formula is then applied to the modified road network.


### Prep workspace

In [1]:
import os, sys
GISFolder = os.getcwd()
GISFolder

'C:\\Users\\wb527163\\GEO-Cdrive-Grace'

In [2]:
# Note: needed to reinstall rtree due to geopandas import error. Did so in the console. 
# conda install -c conda-forge rtree=0.9.3

In [3]:
# load and filter osm network (step 1)
import geopandas as gpd
from geopandas import GeoDataFrame
import pandas as pd
import time
sys.path.append(r"C:\Users\wb527163\.conda\envs\geo\GOSTnets-master")
import GOSTnets as gn

In [4]:
import networkx as nx
import osmnx as ox
import numpy as np
import rasterio as rt
import shapely
from shapely.geometry import Point, box, Polygon
from shapely.ops import unary_union, linemerge, transform
from shapely.wkt import loads
from shapely import wkt
from shapely.geometry import LineString, MultiLineString, Point
import peartree

In [5]:
#### Might not use these
import fiona
from osgeo import gdal
import importlib
import matplotlib.pyplot as plt
import subprocess, glob

In [6]:
from GOSTnets import load_osm as losm
import importlib

In [7]:
pth = os.path.join(GISFolder, "SEN-Cdrive") # Personal folder system for running model.
pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive'

In [8]:
out_pth = os.path.join(GISFolder, "SEN-Cdrive\outputs") # For storing intermediate outputs from the model.
out_pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive\\outputs'

In [9]:
team_pth = 'R:\\SEN\\GEO' # This is where the unmodified input data is stored. Finalized outputs also housed here.
team_pth

'R:\\SEN\\GEO'

### Prepare OSM driving network.
Travel measured in length (meters).

#### Ensure all targeted roads are changed to tertiary. 

In [12]:
flood10 = gpd.read_file("C:/Users/wb527163/GEO-Cdrive-Grace/SEN-Cdrive/scratch.gdb", layer="PFU_1in10")
flood10.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1483825 entries, 0 to 1483824
Data columns (total 4 columns):
 #   Column        Non-Null Count    Dtype   
---  ------        --------------    -----   
 0   PFU_1in10     1483825 non-null  float64 
 1   Shape_Length  1483825 non-null  float64 
 2   Shape_Area    1483825 non-null  float64 
 3   geometry      1483825 non-null  geometry
dtypes: float64(3), geometry(1)
memory usage: 45.3 MB


In [17]:
gTime = nx.read_gpickle("SEN-Cdrive/outputs/gTime_pre-project.pickle")

In [18]:
edges = gn.edge_gdf_from_graph(gTime)

In [13]:
nodes = os.path.join(out_pth, "gTime_node_pre-project.csv")
nodes = pd.read_csv(nodes)
print(nodes.info())
print(edges.info())

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4,5,7,8,10,13,14,15,16,17,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2871182 entries, 0 to 2871181
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   node_ID     int64  
 2   highway     object 
 3   ref         object 
 4   y           float64
 5   x           float64
 6   geometry    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 153.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6203383 entries, 0 to 6203382
Data columns (total 22 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   stnode      int64  
 2   endnode     int64  
 3   length      float64
 4   width       object 
 5   area        object 
 6   highway     object 
 7   ref         object 
 8   maxspeed    object 
 9   time        float64
 10  name        object 
 11  mode        object 
 12  lanes       float64
 13  tunnel      object 
 14  bridge      object 
 15  junction    object 
 16  landuse     object 
 17  a

In [19]:
# Spatial join should be on projected GDFs.
edges = edges.to_crs("EPSG:31028")
flood10 = flood10.to_crs("EPSG:31028")
edges.crs == flood10.crs

True

In [20]:
floodjoin = gpd.sjoin_nearest(edges, flood10, how="left", max_distance=3) 
print(floodjoin.info())
print(floodjoin)

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6377044 entries, 0 to 6203382
Data columns (total 26 columns):
 #   Column        Dtype   
---  ------        -----   
 0   stnode        int64   
 1   endnode       int64   
 2   mode          object  
 3   width         object  
 4   ref           object  
 5   service       object  
 6   junction      object  
 7   bridge        object  
 8   osmid         int64   
 9   oneway        bool    
 10  access        object  
 11  tunnel        object  
 12  Unnamed: 0    int64   
 13  maxspeed      object  
 14  length        float64 
 15  highway       object  
 16  name          object  
 17  landuse       object  
 18  area          object  
 19  time          float64 
 20  lanes         float64 
 21  geometry      geometry
 22  index_right   float64 
 23  PFU_1in10     float64 
 24  Shape_Length  float64 
 25  Shape_Area    float64 
dtypes: bool(1), float64(7), geometry(1), int64(4), object(13)
memory usage: 1.2+ GB
None
     

In [21]:
# How many nodes experienced flooding?
pc_flooded = floodjoin["PFU_1in10"].count() / len(floodjoin) * 100

print("No flood crossing at node:", floodjoin["PFU_1in10"].isnull().sum(), "locations", end="\n\n")
print("Flood crossing at node:", floodjoin["PFU_1in10"].count(), "locations", end="\n\n")
print("\nPercent flooded:", pc_flooded, "percent", "out of", len(floodjoin), "possible locations")

No flood crossing at node: 5919815 locations

Flood crossing at node: 457229 locations


Percent flooded: 7.169920734434323 percent out of 6377044 possible locations


In [23]:
floodjoin = floodjoin[['stnode', 'endnode', 'time', 'length', 'highway', 'osmid', 'geometry', 'PFU_1in10']]
floodjoin = floodjoin.to_crs("EPSG:4326")
floodjoin.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6377044 entries, 0 to 6203382
Data columns (total 8 columns):
 #   Column     Dtype   
---  ------     -----   
 0   stnode     int64   
 1   endnode    int64   
 2   time       float64 
 3   length     float64 
 4   highway    object  
 5   osmid      int64   
 6   geometry   geometry
 7   PFU_1in10  float64 
dtypes: float64(3), geometry(1), int64(3), object(1)
memory usage: 437.9+ MB


In [24]:
# Fewer errors farther down when using dataframe instead of gdf
floodjoin = pd.DataFrame(floodjoin)

In [25]:
# Save progress.
floodjoin.to_csv(os.path.join(out_pth, 'gTime_edge_pre-project_flood10.csv'))

### Create speed penalties.
Note: Flood depths are in centimeters. FATHOM uses meters, but conversion process to vector required some finessing. 

In [10]:
floodjoin = os.path.join(out_pth, "gTime_edge_pre-project_flood10.csv")
floodjoin = pd.read_csv(floodjoin)

In [11]:
# Give a depth to the nodes that don't cross a flood point. 
floodjoin.loc[floodjoin['PFU_1in10'].isnull(), 'PFU_1in10'] = -1
floodjoin["t10"] = 1 # This is the penalty column.
floodjoin.loc[floodjoin['PFU_1in10'] <= 5, 't10'] = 1 # Where no flood crosses, keep the default value (no penalty).
floodjoin.loc[(floodjoin['PFU_1in10'] > 5) & (floodjoin['PFU_1in10'] <= 10), 't10'] = 1.25 # Between 10-30cm flooding, increase travel time by 1.25.
floodjoin.loc[(floodjoin['PFU_1in10'] > 10) & (floodjoin['PFU_1in10'] <= 15), 't10'] = 2
floodjoin.loc[(floodjoin['PFU_1in10'] > 15) & (floodjoin['PFU_1in10'] <= 40), 't10'] = 5
floodjoin.loc[(floodjoin['PFU_1in10'] > 40), 't10'] = 9999
floodjoin

,Unnamed: 0,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in10,t10
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,1.0
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,1.0
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,1.0
3,3,5217543379,358284990,2.385144,33.127,unclassified,59618174,LINESTRING (-12.323682912149085 12.38140360295...,-1.0,1.0
4,4,5217543379,5329928981,1.232928,17.124,unclassified,59618174,LINESTRING (-12.323682912149085 12.38140360295...,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
6377039,6203378,9165665162,9165665161,4.380800,5.476,path,992024621,LINESTRING (-17.484701707370633 14.74572550266...,-1.0,1.0
6377040,6203379,9201905615,9201918817,0.307620,3.418,residential,996565464,LINESTRING (-17.492736907366183 14.73790610266...,-1.0,1.0
6377041,6203380,9201905615,9201918818,0.634050,7.045,residential,996565464,LINESTRING (-17.492736907366183 14.73790610266...,-1.0,1.0
6377042,6203381,9201918818,9201905615,0.634050,7.045,residential,996565464,LINESTRING (-17.492688607366233 14.73786330266...,-1.0,1.0


In [12]:
# Turn the penalty column into a flood-affected time column.
floodjoin['t10'] = floodjoin['t10'] * floodjoin['time']
floodjoin

,Unnamed: 0,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in10,t10
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,2.385144
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,0.769920
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,2.926860
3,3,5217543379,358284990,2.385144,33.127,unclassified,59618174,LINESTRING (-12.323682912149085 12.38140360295...,-1.0,2.385144
4,4,5217543379,5329928981,1.232928,17.124,unclassified,59618174,LINESTRING (-12.323682912149085 12.38140360295...,-1.0,1.232928
...,...,...,...,...,...,...,...,...,...,...
6377039,6203378,9165665162,9165665161,4.380800,5.476,path,992024621,LINESTRING (-17.484701707370633 14.74572550266...,-1.0,4.380800
6377040,6203379,9201905615,9201918817,0.307620,3.418,residential,996565464,LINESTRING (-17.492736907366183 14.73790610266...,-1.0,0.307620
6377041,6203380,9201905615,9201918818,0.634050,7.045,residential,996565464,LINESTRING (-17.492736907366183 14.73790610266...,-1.0,0.634050
6377042,6203381,9201918818,9201905615,0.634050,7.045,residential,996565464,LINESTRING (-17.492688607366233 14.73786330266...,-1.0,0.634050


In [13]:
# Save progress.
floodjoin.to_csv(os.path.join(out_pth, 'gTime_edge_pre-project_flood10.csv'))

In [15]:
print("Travel time without inclement weather\n")
print("Mean:", floodjoin['time'].mean(), end="\n")
print("Median:", floodjoin['time'].median(), end='\n\n')
print("\n\nDuring 1 in 10-year flood conditions\n")
print("Mean:", floodjoin['t10'].mean(), end="\n") 
print("Median:", floodjoin['t10'].median()) 

Travel time without inclement weather

Mean: 7.078734295815471
Median: 3.21903



During 1 in 10-year flood conditions

Mean: 3770.8335061007824
Median: 3.37932


### Convert back to graph object.

In [10]:
# Converting back to graph can cause memory errors. Suggested to restart the kernel and reload the nodes and revised edges at this point.
nodes = os.path.join(out_pth, "gTime_node_pre-project.csv")
nodes = pd.read_csv(nodes)
floodjoin = os.path.join(out_pth, "gTime_edge_pre-project_flood10.csv")
floodjoin = pd.read_csv(floodjoin)
print(nodes.info())
print(floodjoin.info())

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2871182 entries, 0 to 2871181
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   node_ID     int64  
 2   highway     object 
 3   ref         object 
 4   y           float64
 5   x           float64
 6   geometry    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 153.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6377044 entries, 0 to 6377043
Data columns (total 11 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   Unnamed: 0.1  int64  
 2   stnode        int64  
 3   endnode       int64  
 4   time          float64
 5   length        float64
 6   highway       object 
 7   osmid         int64  
 8   geometry      object 
 9   PFU_1in10     float64
 10  t10           float64
dtypes: float64(4), int64(5), object(2)
memory usage: 535.2+ MB
None


In [11]:
print('start: %s\n' % time.ctime())
gTime = gn.edges_and_nodes_gdf_to_graph(nodes, floodjoin, node_tag='node_ID', u_tag='stnode', v_tag='endnode', geometry_tag='geometry')
gn.example_edge(gTime, 10)
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Fri Jan  7 17:04:07 2022

(358284990, 5217543379, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001FFA02FDA60>, 'Unnamed: 0': 0, 'Unnamed: 0.1': 0, 'time': 2.3851440000000004, 'length': 33.127, 'highway': 'unclassified', 'osmid': 59618174, 'PFU_1in10': -1.0, 't10': 2.3851440000000004})
(358284990, 1888282175, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000200059C2A60>, 'Unnamed: 0': 1, 'Unnamed: 0.1': 1, 'time': 0.76992, 'length': 12.832, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in10': -1.0, 't10': 0.76992})
(358284990, 5329792467, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001FFA6E602B0>, 'Unnamed: 0': 2, 'Unnamed: 0.1': 2, 'time': 2.92686, 'length': 48.781, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in10': -1.0, 't10': 2.92686})
(5217543379, 358284990, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001FFA81DCBE0>, 'Unnamed: 0': 3, 'Unnamed: 0.1': 3, 'time': 2.38514400000000

In [12]:
print('start: %s\n' % time.ctime())
gn.save(gTime, 'gTime_pre-project_flood10', out_pth, edges = False, nodes = False)
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Fri Jan  7 17:18:26 2022


end: Fri Jan  7 17:21:08 2022

--- processing complete


### Create travel time values for the road nodes nearest to each service.

Using calculate_OD.

Already measured distance from origin/destination to nearest node in pre-project script.

In [10]:
#%% If starting new session, reload graph from file
gTime = nx.read_gpickle("SEN-Cdrive/outputs/gTime_pre-project_flood10.pickle")

In [11]:
# If starting a new session, load from file.
HDurban_snap = os.path.join(out_pth, "HDurban_snap.csv")
HDurban_snap = pd.read_csv(HDurban_snap)
hamlet_snap = os.path.join(out_pth, "hamlet_snap.csv")
hamlet_snap = pd.read_csv(hamlet_snap)

In [12]:
ag_snap = os.path.join(out_pth, "ag_snap.csv")
ag_snap = pd.read_csv(ag_snap)

In [13]:
HDurban_snap

,Unnamed: 0,Unnamed_ 0,mgrs_code,type,AREA_GEO,SUM,Total_Driv,WP_dens,urb,hd_urb,Urb_class,Shape_Leng,Shape_Area,geometry,NN,NN_dist
0,0,1,28PCU6189_01,bua,27.151434,1.860466e+05,0.000000,6852.183502,1,1,2,0.671746,2.258828e-03,POINT (-16.27318626010657 12.562367084144856),6058226279,2.216606
1,1,8,28PEV0625_01,bua,25.799549,1.064198e+05,0.000000,4124.872441,1,1,2,0.718941,2.149259e-03,POINT (-14.936187613172205 12.892160925397892),6029307183,45.594167
2,2,37,28PBA8597_01,bua,81.242354,3.382677e+05,0.000000,4163.686114,1,1,2,2.303639,6.813610e-03,POINT (-16.987028969206293 14.445279143289168),4998093094,55.383696
3,3,47,28PCA8365_01,bua,42.134069,2.755403e+05,0.000000,6539.609258,1,1,2,1.070058,3.528274e-03,POINT (-16.076128475299285 14.160382012387636),2201506815,55.754910
4,4,56,28PDA4058_01,bua,11.123881,5.186398e+04,0.000000,4662.399756,1,1,2,0.298223,9.311211e-04,POINT (-15.547596052646174 14.100419699131665),4321656809,61.727308
5,5,75,28PBB9234_01,bua,53.453391,2.911360e+05,0.000000,5446.539082,1,1,2,1.468618,4.489114e-03,POINT (-16.926149059908756 14.780621967102865),1697006012,23.047883
6,6,76,28PBB4632_01,bua,226.850348,3.637718e+06,0.000000,16035.762110,1,1,2,4.771359,1.905124e-02,POINT (-17.35638706164378 14.751962825615575),1901689169,5.826006
7,7,80,28PCB0453_01,bua,19.580942,8.712281e+04,0.000000,4449.367491,1,1,2,0.611345,1.645460e-03,POINT (-16.81720508223022 14.94580992504745),6032060028,88.156089
8,8,84,28PCB6720_01,bua,23.634834,1.342721e+05,0.000000,5681.109779,1,1,2,0.689064,1.983306e-03,POINT (-16.227786747221682 14.653718853747844),6040927878,41.930884
9,9,98,28PDB0443_01,bua,196.802191,9.490037e+05,0.000000,2575.818720,1,1,2,3.094788,1.653195e-02,POINT (-15.887758561636817 14.860301673230607),3449495495,28.431974


In [14]:
# We only need to find the origin-destination pairs for nodes closest to the origins and services,
# and some nodes will be the nearest for more than one service (and definitely for multiple origins).
list_hamlet = list(hamlet_snap.NN.unique())
origins = list_hamlet

In [15]:
list_ag = list(ag_snap.NN.unique())
originslist = list_hamlet + list_ag
origins = list(set(originslist))

In [16]:
dests = list(HDurban_snap.NN.unique()) 

In [17]:
len(origins) # 985720 unique nearest nodes.

985720

In [18]:
len(dests) # 58 unique nearest nodes. 

58

In [19]:
gn.example_edge(gTime,10)

(358284990, 5217543379, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001EA1E933AF0>, 'Unnamed: 0': 0, 'Unnamed: 0.1': 0, 'time': 2.3851440000000004, 'length': 33.127, 'highway': 'unclassified', 'osmid': 59618174, 'PFU_1in10': -1.0, 't10': 2.3851440000000004})
(358284990, 1888282175, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001EA1E867DC0>, 'Unnamed: 0': 1, 'Unnamed: 0.1': 1, 'time': 0.76992, 'length': 12.832, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in10': -1.0, 't10': 0.76992})
(358284990, 5329792467, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001EA16A68FD0>, 'Unnamed: 0': 2, 'Unnamed: 0.1': 2, 'time': 2.92686, 'length': 48.781, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in10': -1.0, 't10': 2.92686})
(5217543379, 358284990, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001EA16A544F0>, 'Unnamed: 0': 3, 'Unnamed: 0.1': 3, 'time': 2.3851440000000004, 'length': 33.127, 'highway': 

calculate_OD won't run if any of the edge times are null or zero. If there are any hits, reassign time values for those edges to a very small time. A more efficient way to do this would be to simplify junctions with the clean_network() tool, but that was throwing errors.

In [20]:
edges = gn.edge_gdf_from_graph(gTime)
len(edges.loc[edges['t10'].isnull()])

0

In [21]:
len(edges.loc[edges['t10']==0]) 

0

In [22]:
fail_value = 999999999 # If there is no shortest path, the OD pair will be assigned the fail value.

In [23]:
print('start: %s\n' % time.ctime())
OD = gn.calculate_OD(gTime, origins, dests, fail_value, weight = 't10')
# Takes a few minutes.
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

In [24]:
OD_df = pd.DataFrame(OD, index = origins, columns = dests)

In [25]:
OD_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985720 entries, 3331684166 to 3706826877
Data columns (total 58 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   6058226279  985720 non-null  float64
 1   6029307183  985720 non-null  float64
 2   4998093094  985720 non-null  float64
 3   2201506815  985720 non-null  float64
 4   4321656809  985720 non-null  float64
 5   1697006012  985720 non-null  float64
 6   1901689169  985720 non-null  float64
 7   6032060028  985720 non-null  float64
 8   6040927878  985720 non-null  float64
 9   3449495495  985720 non-null  float64
 10  3990543961  985720 non-null  float64
 11  8972391475  985720 non-null  float64
 12  3418418812  985720 non-null  float64
 13  1983641803  985720 non-null  float64
 14  6014451367  985720 non-null  float64
 15  6027163276  985720 non-null  float64
 16  2833577858  985720 non-null  float64
 17  7321088861  985720 non-null  float64
 18  6045659373  985720 non-null  fl

In [26]:
OD_df.tail()

,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
8925478824,44143.384955,170750.964644,5946.118129,8520.321136,33215.724688,3586.021988,6073.297937,2932.849334,5697.346178,5024.173854,...,6431.994008,9772.125246,37027.586226,37009.987464,6094.891354,5658.806252,5627.110420,5621.858030,5150.677646,5210.615264
3706826868,11646.839829,150491.899528,42593.953030,35174.609681,6111.195684,40237.261033,42663.363877,39202.885121,34138.878458,32269.311484,...,39727.768877,43067.900115,17817.263688,17799.664926,42684.957294,42248.872192,42217.176360,42211.923970,32716.881405,32656.769215
3706826871,11584.582437,150429.642136,42531.695638,35112.352289,6173.453076,40175.003641,42601.106485,39140.627729,34076.621066,32207.054092,...,39665.511485,43005.642723,17755.006296,17737.407534,42622.699902,42186.614800,42154.918968,42149.666578,32654.624013,32594.511823
3706826876,11657.735373,150502.795072,42604.848574,35185.505225,6100.300140,40248.156577,42674.259421,39213.780665,34149.774002,32280.207028,...,39738.664421,43078.795659,17828.159232,17810.560470,42695.852838,42259.767736,42228.071904,42222.819514,32727.776949,32667.664759
3706826877,11599.697685,150444.757384,42546.810886,35127.467537,6158.337828,40190.118889,42616.221733,39155.742977,34091.736314,32222.169340,...,39680.626733,43020.757971,17770.121544,17752.522782,42637.815150,42201.730048,42170.034216,42164.781826,32669.739261,32609.627071


In [27]:
# Convert to minutes and save to file.
OD_min = OD_df[OD_df <fail_value] / 60
OD_min.to_csv(os.path.join(out_pth, 'OD_allorigins_pre-project_flood10.csv'))
OD_min

,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
3331684166,712.885186,2823.011514,246.631329,10.569381,530.757515,207.296393,248.750992,198.063276,60.029574,97.714216,...,221.849165,277.518019,594.288540,593.995228,249.110882,241.842797,241.314533,241.226994,96.444984,107.620912
3571449893,216.749440,2386.130808,733.514641,609.858919,170.668947,694.236441,734.671489,676.996843,592.596732,561.437282,...,685.744905,741.413759,174.886877,174.593564,735.031379,727.763294,727.235030,727.147490,568.896781,567.894911
3571449966,217.136351,2386.517719,733.901553,610.245830,171.055859,694.623353,735.058400,677.383754,592.983643,561.824194,...,686.131817,741.800671,175.273788,174.980476,735.418291,728.150206,727.621942,727.534402,569.283692,568.281823
3405774993,88.160619,2520.464644,745.743296,622.087573,160.299823,706.465096,746.900143,689.225497,604.825386,573.665937,...,697.973560,753.642414,309.220713,308.927401,747.260034,739.991949,739.463685,739.376145,581.125435,580.123566
3405774994,88.561368,2520.063895,745.342547,621.686824,159.899074,706.064347,746.499394,688.824748,604.424637,573.265188,...,697.572811,753.241665,308.819964,308.526652,746.859285,739.591200,739.062936,738.975396,580.724686,579.722817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8925478824,735.723083,2845.849411,99.101969,142.005352,553.595411,59.767033,101.221632,48.880822,94.955770,83.736231,...,107.199900,162.868754,617.126437,616.833124,101.581523,94.313438,93.785174,93.697634,85.844627,86.843588
3706826868,194.113997,2508.198325,709.899217,586.243495,101.853261,670.621017,711.056065,653.381419,568.981308,537.821858,...,662.129481,717.798335,296.954395,296.661082,711.415955,704.147870,703.619606,703.532066,545.281357,544.279487
3706826871,193.076374,2507.160702,708.861594,585.205871,102.890885,669.583394,710.018441,652.343795,567.943684,536.784235,...,661.091858,716.760712,295.916772,295.623459,710.378332,703.110247,702.581983,702.494443,544.243734,543.241864
3706826876,194.295590,2508.379918,710.080810,586.425087,101.671669,670.802610,711.237657,653.563011,569.162900,538.003450,...,662.311074,717.979928,297.135987,296.842675,711.597547,704.329462,703.801198,703.713659,545.462949,544.461079


In [28]:
print(OD_min.isna().sum())

6058226279    3
6029307183    3
4998093094    3
2201506815    3
4321656809    3
1697006012    3
1901689169    3
6032060028    3
6040927878    3
3449495495    3
3990543961    3
8972391475    3
3418418812    3
1983641803    3
6014451367    3
6027163276    3
2833577858    3
7321088861    3
6045659373    3
5528866190    3
6027939517    3
6031093257    3
1883155712    3
6026417971    3
6031245697    3
7493947593    3
1859359090    3
6053816462    3
6027530676    3
2988435647    3
7838754747    3
6053825179    3
2988435792    3
3036877534    3
6056472223    3
7460241128    3
6028595154    3
4053739078    3
4071261250    3
6014815775    3
6024430122    3
8229317581    3
6027834438    3
6024894515    3
6024894530    3
6024894513    3
8169321141    3
8200304626    3
1968458114    3
9359670710    3
6021177135    3
6027615161    3
6027276892    3
6041228287    3
5536661253    3
7357630367    3
8178147277    3
6026834850    3
dtype: int64


In [29]:
# Create origin-specific matrix and save to file.
OD_hamlet = OD_df.loc[list_hamlet,:]
OD_hamlet = OD_hamlet[OD_hamlet < fail_value] / 60 
OD_hamlet.to_csv(os.path.join(out_pth, 'OD_hamlet_pre-project_flood10.csv'))
OD_hamlet

,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
7761872870,63.944594,2661.109731,886.388383,762.732660,300.944910,847.110183,887.545230,829.870584,745.470473,714.311023,...,838.618647,894.287501,449.865800,449.572487,887.905120,880.637035,880.108771,880.021232,721.770522,720.768652
7761872869,63.926035,2661.091171,886.369823,762.714101,300.926350,847.091623,887.526670,829.852025,745.451913,714.292464,...,838.600087,894.268941,449.847241,449.553928,887.886561,880.618476,880.090212,880.002672,721.751963,720.750093
6442044321,62.916203,2660.081339,885.359991,761.704269,299.916518,846.081791,886.516838,828.842193,744.442081,713.282632,...,837.590255,893.259109,448.837409,448.544096,886.876729,879.608644,879.080380,878.992840,720.742131,719.740261
2142496418,64.369725,2661.534861,886.813513,763.157791,301.370040,847.535313,887.970360,830.295715,745.895603,714.736154,...,839.043777,894.712631,450.290931,449.997618,888.330251,881.062166,880.533902,880.446362,722.195653,721.193783
2142496429,64.752099,2661.917235,887.195887,763.540165,301.752414,847.917687,888.352734,830.678089,746.277977,715.118528,...,839.426151,895.095005,450.673305,450.379992,888.712625,881.444540,880.916276,880.828736,722.578027,721.576157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9208004175,177445.540717,179555.667046,176832.674061,176878.793338,177263.413046,176793.395861,176827.212158,176776.156263,176831.743755,176793.553866,...,176679.123234,176622.095460,177326.944072,177326.650759,176834.190799,176815.557740,176814.667862,176814.561084,176795.662262,176796.661223
3884280043,993.922236,2928.538223,435.149435,441.072055,811.794565,395.871235,436.306282,378.631636,394.022472,355.531399,...,278.242294,257.235026,633.873551,633.580238,436.666173,429.398088,428.869824,428.782284,364.724199,357.764104
3884280001,987.060009,2921.675996,428.287208,434.209828,804.932338,389.009008,429.444055,371.769409,387.160245,348.669172,...,271.380067,250.372799,627.011324,626.718011,429.803946,422.535861,422.007597,421.920057,357.861972,350.901877
8592243457,962.853587,2915.269535,404.080786,410.003406,780.725916,364.802586,405.237633,347.562987,362.953823,324.462750,...,247.173644,226.166377,620.604862,620.311550,405.597523,398.329438,397.801175,397.713635,333.655549,326.695455


In [30]:
print(OD_hamlet.isna().sum())

6058226279    1
6029307183    1
4998093094    1
2201506815    1
4321656809    1
1697006012    1
1901689169    1
6032060028    1
6040927878    1
3449495495    1
3990543961    1
8972391475    1
3418418812    1
1983641803    1
6014451367    1
6027163276    1
2833577858    1
7321088861    1
6045659373    1
5528866190    1
6027939517    1
6031093257    1
1883155712    1
6026417971    1
6031245697    1
7493947593    1
1859359090    1
6053816462    1
6027530676    1
2988435647    1
7838754747    1
6053825179    1
2988435792    1
3036877534    1
6056472223    1
7460241128    1
6028595154    1
4053739078    1
4071261250    1
6014815775    1
6024430122    1
8229317581    1
6027834438    1
6024894515    1
6024894530    1
6024894513    1
8169321141    1
8200304626    1
1968458114    1
9359670710    1
6021177135    1
6027615161    1
6027276892    1
6041228287    1
5536661253    1
7357630367    1
8178147277    1
6026834850    1
dtype: int64


In [31]:
# Create origin-specific matrix and save to file.
OD_ag = OD_df.loc[list_ag,:]
OD_ag = OD_ag[OD_ag < fail_value] / 60 
OD_ag.to_csv(os.path.join(out_pth, 'OD_ag_pre-project_flood10.csv'))
OD_ag

,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
3507831609,83.755636,2682.494783,907.773435,784.117712,322.329962,868.495235,908.930282,851.255636,766.855525,735.696076,...,860.003699,915.672553,471.250852,470.957540,909.290172,902.022087,901.493824,901.406284,743.155574,742.153704
3507831510,90.114132,2688.853279,914.131931,790.476209,328.688458,874.853731,915.288779,857.614133,773.214022,742.054572,...,866.362195,922.031049,477.609349,477.316036,915.648669,908.380584,907.852320,907.764780,749.514071,748.512201
6188134127,25.237636,2623.976783,849.255434,725.599712,263.811962,809.977235,850.412282,792.737636,708.337525,677.178075,...,801.485699,857.154553,412.732852,412.439539,850.772172,843.504087,842.975823,842.888283,684.637574,683.635704
8631201421,722.161882,2169.322786,1057.566861,933.911139,655.162619,1018.288662,1058.723709,1001.049063,916.648952,885.489502,...,1009.797126,1065.465980,425.374472,425.081159,1059.083599,1051.815514,1051.287250,1051.199710,892.949001,891.947131
8598305977,722.607483,2169.768387,1058.012463,934.356741,655.608221,1018.734263,1059.169310,1001.494665,917.094554,885.935104,...,1010.242727,1065.911581,425.820073,425.526761,1059.529201,1052.261116,1051.732852,1051.645312,893.394603,892.392733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651042474,174281.469685,176391.596013,173668.603029,173714.722305,174099.342014,173629.324829,173663.141125,173612.085230,173667.672722,173629.482833,...,173515.052202,173458.024427,174162.873039,174162.579727,173670.119766,173651.486707,173650.596829,173650.490051,173631.591230,173632.590190
3651042508,174273.319430,176383.445758,173660.452774,173706.572050,174091.191759,173621.174574,173654.990870,173603.934975,173659.522467,173621.332578,...,173506.901947,173449.874172,174154.722784,174154.429472,173661.969512,173643.336452,173642.446574,173642.339797,173623.440975,173624.439935
3651042501,174278.814545,176388.940874,173665.947889,173712.067166,174096.686874,173626.669689,173660.485986,173609.430091,173665.017583,173626.827694,...,173512.397062,173455.369288,174160.217900,174159.924587,173667.464627,173648.831568,173647.941690,173647.834912,173628.936090,173629.935051
3651042393,174265.284243,176375.410571,173652.417587,173698.536863,174083.156572,173613.139387,173646.955684,173595.899789,173651.487281,173613.297391,...,173498.866760,173441.838985,174146.687598,174146.394285,173653.934325,173635.301266,173634.411387,173634.304610,173615.405788,173616.404748


In [32]:
print(OD_ag.isna().sum())

6058226279    2
6029307183    2
4998093094    2
2201506815    2
4321656809    2
1697006012    2
1901689169    2
6032060028    2
6040927878    2
3449495495    2
3990543961    2
8972391475    2
3418418812    2
1983641803    2
6014451367    2
6027163276    2
2833577858    2
7321088861    2
6045659373    2
5528866190    2
6027939517    2
6031093257    2
1883155712    2
6026417971    2
6031245697    2
7493947593    2
1859359090    2
6053816462    2
6027530676    2
2988435647    2
7838754747    2
6053825179    2
2988435792    2
3036877534    2
6056472223    2
7460241128    2
6028595154    2
4053739078    2
4071261250    2
6014815775    2
6024430122    2
8229317581    2
6027834438    2
6024894515    2
6024894530    2
6024894513    2
8169321141    2
8200304626    2
1968458114    2
9359670710    2
6021177135    2
6027615161    2
6027276892    2
6041228287    2
5536661253    2
7357630367    2
8178147277    2
6026834850    2
dtype: int64


### Filter 1st nearest

#### Check each file to make sure nearest neighbor column is named correctly. If not, rename.

In [10]:
# Reload from file even if already loaded. Quickest way to ensure NN is a column rather than only the index.
OD_hamlet = os.path.join(out_pth, "OD_hamlet_pre-project_flood10.csv")
OD_hamlet = pd.read_csv(OD_hamlet)

In [11]:
OD_ag = os.path.join(out_pth, "OD_ag_pre-project_flood10.csv")
OD_ag = pd.read_csv(OD_ag)

In [12]:
OD_ag.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) 

In [13]:
OD_hamlet.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) 

In [14]:
OD_ag

,NN,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
0,3507831609,83.755636,2682.494783,907.773435,784.117712,322.329962,868.495235,908.930282,851.255636,766.855525,...,860.003699,915.672553,471.250852,470.957540,909.290172,902.022087,901.493824,901.406284,743.155574,742.153704
1,3507831510,90.114132,2688.853279,914.131931,790.476209,328.688458,874.853731,915.288779,857.614133,773.214022,...,866.362195,922.031049,477.609349,477.316036,915.648669,908.380584,907.852320,907.764780,749.514071,748.512201
2,6188134127,25.237636,2623.976783,849.255434,725.599712,263.811962,809.977235,850.412282,792.737636,708.337525,...,801.485699,857.154553,412.732852,412.439539,850.772172,843.504087,842.975823,842.888283,684.637574,683.635704
3,8631201421,722.161882,2169.322786,1057.566861,933.911139,655.162619,1018.288662,1058.723709,1001.049063,916.648952,...,1009.797126,1065.465980,425.374472,425.081159,1059.083599,1051.815514,1051.287250,1051.199710,892.949001,891.947131
4,8598305977,722.607483,2169.768387,1058.012463,934.356741,655.608221,1018.734263,1059.169310,1001.494665,917.094554,...,1010.242727,1065.911581,425.820073,425.526761,1059.529201,1052.261116,1051.732852,1051.645312,893.394603,892.392733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967547,3651042474,174281.469685,176391.596013,173668.603029,173714.722305,174099.342014,173629.324829,173663.141125,173612.085230,173667.672722,...,173515.052202,173458.024427,174162.873039,174162.579727,173670.119766,173651.486707,173650.596829,173650.490051,173631.591230,173632.590190
967548,3651042508,174273.319430,176383.445758,173660.452774,173706.572050,174091.191759,173621.174574,173654.990870,173603.934975,173659.522467,...,173506.901947,173449.874172,174154.722784,174154.429472,173661.969512,173643.336452,173642.446574,173642.339797,173623.440975,173624.439935
967549,3651042501,174278.814545,176388.940874,173665.947889,173712.067166,174096.686874,173626.669689,173660.485986,173609.430091,173665.017583,...,173512.397062,173455.369288,174160.217900,174159.924587,173667.464627,173648.831568,173647.941690,173647.834912,173628.936090,173629.935051
967550,3651042393,174265.284243,176375.410571,173652.417587,173698.536863,174083.156572,173613.139387,173646.955684,173595.899789,173651.487281,...,173498.866760,173441.838985,174146.687598,174146.394285,173653.934325,173635.301266,173634.411387,173634.304610,173615.405788,173616.404748


In [15]:
OD_hamlet

,NN,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
0,7761872870,63.944594,2661.109731,886.388383,762.732660,300.944910,847.110183,887.545230,829.870584,745.470473,...,838.618647,894.287501,449.865800,449.572487,887.905120,880.637035,880.108771,880.021232,721.770522,720.768652
1,7761872869,63.926035,2661.091171,886.369823,762.714101,300.926350,847.091623,887.526670,829.852025,745.451913,...,838.600087,894.268941,449.847241,449.553928,887.886561,880.618476,880.090212,880.002672,721.751963,720.750093
2,6442044321,62.916203,2660.081339,885.359991,761.704269,299.916518,846.081791,886.516838,828.842193,744.442081,...,837.590255,893.259109,448.837409,448.544096,886.876729,879.608644,879.080380,878.992840,720.742131,719.740261
3,2142496418,64.369725,2661.534861,886.813513,763.157791,301.370040,847.535313,887.970360,830.295715,745.895603,...,839.043777,894.712631,450.290931,449.997618,888.330251,881.062166,880.533902,880.446362,722.195653,721.193783
4,2142496429,64.752099,2661.917235,887.195887,763.540165,301.752414,847.917687,888.352734,830.678089,746.277977,...,839.426151,895.095005,450.673305,450.379992,888.712625,881.444540,880.916276,880.828736,722.578027,721.576157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74657,9208004175,177445.540717,179555.667046,176832.674061,176878.793338,177263.413046,176793.395861,176827.212158,176776.156263,176831.743755,...,176679.123234,176622.095460,177326.944072,177326.650759,176834.190799,176815.557740,176814.667862,176814.561084,176795.662262,176796.661223
74658,3884280043,993.922236,2928.538223,435.149435,441.072055,811.794565,395.871235,436.306282,378.631636,394.022472,...,278.242294,257.235026,633.873551,633.580238,436.666173,429.398088,428.869824,428.782284,364.724199,357.764104
74659,3884280001,987.060009,2921.675996,428.287208,434.209828,804.932338,389.009008,429.444055,371.769409,387.160245,...,271.380067,250.372799,627.011324,626.718011,429.803946,422.535861,422.007597,421.920057,357.861972,350.901877
74660,8592243457,962.853587,2915.269535,404.080786,410.003406,780.725916,364.802586,405.237633,347.562987,362.953823,...,247.173644,226.166377,620.604862,620.311550,405.597523,398.329438,397.801175,397.713635,333.655549,326.695455


#### Find first, second, and third nearest destination for each origin node. 

In [16]:
fail_value = 999999999

In [17]:
# Nearest
OD_ag["ag_HD1"] = 0
sub = OD_ag.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
OD_ag["ag_HD1"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
ag1 = OD_ag[['NN', 'ag_HD1']] # Remove unnecessary OD values.


# Second nearest
dupes = OD_ag.apply(pd.Series.duplicated, axis = 1, keep=False) # If a number is repeated within a row, value is True. If not, False.
# The first time this is done, there should be two True values per row, unless any POIs are equidistant.
dupes = OD_ag.where(~dupes, fail_value) # For any value that appears more than once in its row, it is replaced with the fail_value.
OD_ag["ag_HD2"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column. No need to filter 1st nearest as its new "dupes" value is too high to be caught.
OD_ag["ag_HD2"] = Dsub.min(axis=1) 
ag2 = OD_ag.loc[:,['NN', 'ag_HD2']] 


# Third nearest
dupes = OD_ag.apply(pd.Series.duplicated, axis = 1, keep=False)
# Since this includes both first and second nearest columns, there should be four True values per row, unless POIs are equidistant.
dupes = OD_ag.where(~dupes, fail_value)

OD_ag["ag_HD3"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column.
OD_ag["ag_HD3"] = Dsub.min(axis=1)
ag3 = OD_ag.loc[:,['NN', 'ag_HD3']]

# Combine and write to file
ag_all = OD_ag.loc[:,['NN', 'ag_HD1', 'ag_HD2', 'ag_HD3']]
ag_all.to_csv(os.path.join(out_pth, 'ag_to_HDurban_drive_pre-project_flood10.csv'))
ag_all.head()

,NN,ag_HD1,ag_HD2,ag_HD3
0,3507831609,82.054823,83.755636,322.329962
1,3507831510,88.413319,90.114132,328.688458
2,6188134127,23.536822,25.237636,263.811962
3,8631201421,419.573315,421.041963,425.081159
4,8598305977,420.018916,421.487564,425.526761


In [18]:
# Nearest
OD_hamlet["ha_HD1"] = 0
sub = OD_hamlet.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
OD_hamlet["ha_HD1"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
hamlet1 = OD_hamlet[['NN', 'ha_HD1']] # Remove unnecessary OD values.


# Second nearest
dupes = OD_hamlet.apply(pd.Series.duplicated, axis = 1, keep=False) # If a number is repeated within a row, value is True. If not, False.
# The first time this is done, there should be two True values per row, unless any POIs are equidistant.
dupes = OD_hamlet.where(~dupes, fail_value) # For any value that appears more than once in its row, it is replaced with the fail_value.
OD_hamlet["ha_HD2"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column. No need to filter 1st nearest as its new "dupes" value is too high to be caught.
OD_hamlet["ha_HD2"] = Dsub.min(axis=1) 
hamlet2 = OD_hamlet.loc[:,['NN', 'ha_HD2']] 


# Third nearest
dupes = OD_hamlet.apply(pd.Series.duplicated, axis = 1, keep=False)
# Since this includes both first and second nearest columns, there should be four True values per row, unless POIs are equidistant.
dupes = OD_hamlet.where(~dupes, fail_value)
OD_hamlet["ha_HD3"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column.
OD_hamlet["ha_HD3"] = Dsub.min(axis=1)
hamlet3 = OD_hamlet.loc[:,['NN', 'ha_HD3']]


# Combine and write to file
hamlet_all = OD_hamlet.loc[:,['NN', 'ha_HD1', 'ha_HD2', 'ha_HD3']]
hamlet_all.to_csv(os.path.join(out_pth, 'hamlet_to_HDurban_drive_pre-project_flood10.csv'))
hamlet_all.head()

,NN,ha_HD1,ha_HD2,ha_HD3
0,7761872870,63.944594,67.312774,300.944910
1,7761872869,63.926035,67.294215,300.926350
2,6442044321,62.916203,66.284383,299.916518
3,2142496418,64.369725,67.737905,301.370040
4,2142496429,64.752099,68.120279,301.752414


### End of script. Load into QGIS or Arc and visualize at 10 min intervals. 
QML file for symbology in QGIS:
R:\GEOGlobal\Design\symb_traveltimes_10min.qml